In [5]:
import numpy as np

A = np.random.rand(4,4)
print("A\n", A)

I = A.argsort(axis=1)
print(I)

R = A[np.arange(np.shape(A)[0])[:,np.newaxis], I]

print(R)

S = list(zip(R.ravel(), I.ravel()))

def chunks(l,n):
    return [l[i:i+n] for i in range(0, len(l), n) ]

chunks(S, I.shape[1])

#.reshape(I.shape)




A
 [[0.18528868 0.6433512  0.38171788 0.14222304]
 [0.42198118 0.65129068 0.86281611 0.73881868]
 [0.9370838  0.10895964 0.49278145 0.04611704]
 [0.46035037 0.14642563 0.17458223 0.75107309]]
[[3 0 2 1]
 [0 1 3 2]
 [3 1 2 0]
 [1 2 0 3]]
[[0.14222304 0.18528868 0.38171788 0.6433512 ]
 [0.42198118 0.65129068 0.73881868 0.86281611]
 [0.04611704 0.10895964 0.49278145 0.9370838 ]
 [0.14642563 0.17458223 0.46035037 0.75107309]]


[[(0.14222304428971566, 3),
  (0.185288684104153, 0),
  (0.381717882447678, 2),
  (0.6433511962022134, 1)],
 [(0.42198117610503916, 0),
  (0.6512906770638834, 1),
  (0.7388186753779015, 3),
  (0.862816109271107, 2)],
 [(0.0461170373026788, 3),
  (0.10895964424448024, 1),
  (0.4927814523656405, 2),
  (0.9370837955347795, 0)],
 [(0.1464256323395947, 1),
  (0.1745822277881084, 2),
  (0.460350369122247, 0),
  (0.7510730889718991, 3)]]

In [56]:
# Python program explaining 
# argpartition() function 
  
import numpy as geek 
  
# input 2d array 
in_arr =  geek.array([
    [ 0,  1,  2,  3],
    [ 4,  5,  6,  7],
    [ 8,  9, 11, 10],
    [12, 13, 20, 15]
])
print ("Input array : ", in_arr)  
  
# output sorted array indices 
out_arr1 = geek.argsort(in_arr, kind ='mergesort', axis = 0) 
print ("Output sorteded array indices along axis 0: ", out_arr1) 
out_arr2 = geek.argsort(in_arr, kind ='heapsort', axis = 1) 
print ("Output sorteded array indices along axis 1: ", out_arr2) 
in_arr[out_arr2]

Input array :  [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 11 10]
 [12 13 20 15]]
Output sorteded array indices along axis 0:  [[0 0 0 0]
 [1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]]
Output sorteded array indices along axis 1:  [[0 1 2 3]
 [0 1 2 3]
 [0 1 3 2]
 [0 1 3 2]]


array([[[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 11, 10],
        [12, 13, 20, 15]],

       [[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 11, 10],
        [12, 13, 20, 15]],

       [[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [12, 13, 20, 15],
        [ 8,  9, 11, 10]],

       [[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [12, 13, 20, 15],
        [ 8,  9, 11, 10]]])

In [1]:
import pypclda
import tests
import tests.fixture as fixture
import pypclda.config_default as config_default
import jpype
import jpype.imports
from jpype.types import *

cc = jpype.JPackage("cc")
java = jpype.JPackage("java")
lda_util = pypclda.get_util()


In [33]:
import os
import math
import pytest
import logging
import unittest
import types
import pypclda
import pypclda.config_default as config_default
import tests.fixture as fixture
import numpy as np
import toolz

PCPLDA_JAR_PATH = os.path.join(os.getcwd(), "lib", "PCPLDA-8.5.1.jar")

import jpype
import jpype.imports
from jpype.types import *

cc = jpype.JPackage("cc")
java = jpype.JPackage("java")

def calculate_top_relevance_tokens(n_top_tokens, type_topic_counts_matrix, beta, _lambda, alphabet):

    n_topics = len(type_topic_counts_matrix[0])
    n_types  = len(type_topic_counts_matrix)
    n_top_tokens = min(n_top_tokens, n_types)

    relevance_matrix = np.array(
        pypclda.calculate_token_relevance_matrix(type_topic_counts_matrix, beta, _lambda)
    )
    sorted_indices = relevance_matrix.argsort(axis=1)

    sorted_matrix = relevance_matrix[
        np.arange(np.shape(relevance_matrix)[0])[:,np.newaxis],
        sorted_indices
    ]

    sliced_indices = np.flip(sorted_indices[:,-n_top_tokens:], axis=1)
    sliced_matrix = np.flip(sorted_matrix[:,-n_top_tokens:], axis=1)

    descending_token_relevances = (
        (str(alphabet.lookupObject(w[1])), w[0], i)
            for i, w in enumerate(zip(
                    sliced_matrix.ravel(),
                    sliced_indices.ravel()
                ))
    )
    return [
        [ w for w in row ]
            for row in toolz.partition(n_top_tokens, descending_token_relevances)
    ]


In [34]:
#  Arrange
n_top_tokens = 20

lda_util = cc.mallet.util.LDAUtils()
config = fixture.fixture_lda_config()
sampler = fixture.fixture_sampler()
beta = config.getBeta(config_default.BETA_DEFAULT)
type_topic_counts = pypclda.get_token_topic_matrix(sampler)
n_types = len(type_topic_counts)
n_topics = len(type_topic_counts[0])
v_lambda = config.getLambda(config_default.LAMBDA_DEFAULT)
alphabet = sampler.getAlphabet()

# Act
top_token_relevance_python = calculate_top_relevance_tokens(
    n_top_tokens, type_topic_counts, beta, v_lambda, alphabet
)

# Assert
top_token_relevance_java = lda_util.getTopRelevanceWords(
    n_top_tokens, n_types, n_topics, type_topic_counts,
    beta, v_lambda, sampler.getAlphabet()
)

python_words = [
    [ w[0] for w in row ]
        for row in top_token_relevance_python
]

top_token_relevance_java = [ [str(w) for w in ws] for ws in top_token_relevance_java ]


In [53]:
import pandas as pd

df_java = pd.DataFrame(data=top_token_relevance_java, dtype=str)
df_python =  pd.DataFrame(data=top_token_relevance_python, dtype=str)

df = df_java.append(df_python)

display(df.loc[2])


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
2,full,presence,hot-headed,thus,length,slower,wondering,fellow,meet,local,fluff,done,interesting,close-ups,editing,quick-paced,rooms,brooks,beauty,staying
2,"(fluff, -2.8738417734110673, 40)","(quick-paced, -2.8738417734110673, 41)","(slower, -2.8738417734110673, 42)","(hot-headed, -2.8738417734110673, 43)","(wondering, -2.8738417734110673, 44)","(length, -2.8738417734110673, 45)","(presence, -2.8738417734110673, 46)","(meet, -2.8738417734110673, 47)","(done, -2.8738417734110673, 48)","(local, -2.8738417734110673, 49)","(rooms, -2.8738417734110673, 50)","(editing, -2.8738417734110673, 51)","(close-ups, -2.8738417734110673, 52)","(interesting, -2.8738417734110673, 53)","(full, -2.8738417734110673, 54)","(fellow, -2.8738417734110673, 55)","(thus, -2.8738417734110673, 56)","(beauty, -2.9159859796741974, 57)","(staying, -2.9159859796741974, 58)","(brooks, -2.9159859796741974, 59)"
